In [2]:

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier



$r = \text{NBR_cas}''(t)$

$AVG\ P(r>0)$ sur 14 J

$AVG\ P(r==0)$ sur 14 J

$AVG\ P(r<0)$ sur 14 J

Output $\rightarrow$ \[-1, 1\]

ECART-type $\rightarrow$ certitude

In [3]:
DATA_PATH = "../../data_processed/usa_urban_final.csv"
df = pd.read_csv(DATA_PATH, index_col="Unnamed: 0")

df.drop(labels=['smoothed_cumul', 'smoothed_d1'], axis=1, inplace=True)

df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)

df = df.sort_values(['shapeID', 'date'])

df

,shapeID,date,mobility_transit_stations,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_residential,mobility_workplaces,smoothed_d2,x0_mean,...,x7_amax,x8_mean,x8_amin,x8_amax,x9_mean,x9_amin,x9_amax,x10_mean,x10_amin,x10_amax
12582,10003,2020-03-29,-42.0,-28.0,-6.0,2.0,17.0,-39.0,3.880952,-0.815314,...,0.762621,-0.290963,-0.987737,0.032615,-0.376480,-0.833365,0.210322,0.481506,-0.215669,1.328427
12583,10003,2020-03-30,-39.0,-26.0,0.0,49.0,17.0,-40.0,2.785714,-1.417909,...,0.860248,0.550598,-0.238915,1.072211,-0.201743,-0.840541,0.407437,0.070182,-0.885137,0.932346
12584,10003,2020-03-31,-40.0,-35.0,-9.0,78.0,11.0,-25.0,2.642857,0.907826,...,1.432990,-0.418361,-1.100802,0.766658,0.034947,-0.495268,0.470400,-0.747065,-1.050835,-0.306901
12585,10003,2020-04-01,-56.0,-38.0,-19.0,17.0,10.0,-30.0,11.309524,1.074053,...,0.858769,-1.154608,-1.715199,-0.941215,0.829711,0.320043,1.218661,-0.731472,-1.216402,0.030832
12586,10003,2020-04-02,-52.0,-31.0,-12.0,-49.0,19.0,-45.0,11.071429,1.614494,...,1.974859,-0.561181,-1.704962,0.777859,0.254482,-0.507295,0.935299,-0.212581,-1.210952,0.418377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29557,55133,2020-11-16,-31.0,-23.0,-9.0,98.0,14.0,-35.0,245.380952,0.374379,...,0.245586,0.285412,-0.105664,0.863202,-0.217611,-0.801242,0.307856,0.241920,-0.126822,0.736519
29558,55133,2020-11-17,-34.5,-16.0,-1.0,144.0,4.0,-6.0,179.880952,-0.200847,...,-0.812017,0.697624,0.464090,0.950370,0.101097,-0.132406,0.507043,0.568914,0.204780,0.828934
29559,55133,2020-11-18,-38.0,-17.0,-9.0,114.0,3.0,-10.0,-96.523810,0.671859,...,-0.587159,0.753704,0.416389,1.219656,0.332824,-0.363286,0.880683,0.157192,-0.286517,0.703741
29560,55133,2020-11-19,-26.0,-17.0,-3.0,74.0,12.0,-32.0,-322.095238,1.186025,...,-0.314402,0.924094,0.587659,1.672445,0.053305,-0.199086,0.431580,-0.046998,-0.446054,0.140200


In [6]:
df.describe(percentiles=[0.33,0.66])

,shapeID,mobility_transit_stations,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_residential,mobility_workplaces,smoothed_d2,x0_mean,x0_amin,...,x7_amax,x8_mean,x8_amin,x8_amax,x9_mean,x9_amin,x9_amax,x10_mean,x10_amin,x10_amax
count,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,...,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000,34521.000000
mean,31559.012833,-31.787717,-22.116997,-6.993742,51.424385,11.353976,-34.656539,0.663647,-0.900682,-1.952420,...,0.281898,0.075240,-0.338583,0.537464,0.037097,-0.422045,0.481167,-0.279378,-0.742525,0.215226
std,12914.751288,22.115407,15.686999,9.908407,70.324777,6.898706,15.197313,36.716786,1.691739,1.619294,...,0.972265,0.736304,0.759404,0.815128,0.698473,0.793417,0.807862,0.631255,0.640753,0.759986
min,10003.000000,-89.000000,-89.000000,-63.000000,-86.000000,-7.000000,-83.000000,-1597.738095,-17.742811,-19.162831,...,-2.206789,-3.211431,-4.438735,-2.638421,-2.814116,-4.285868,-2.554283,-2.633347,-3.427467,-2.387668
33%,24510.000000,-42.000000,-25.000000,-10.000000,13.000000,8.000000,-41.000000,-2.785714,-1.752812,-2.749990,...,-0.238520,-0.260865,-0.666161,0.157100,-0.238610,-0.715788,0.173939,-0.534585,-1.003849,-0.087902
50%,34017.000000,-32.000000,-19.000000,-7.000000,41.000000,11.000000,-36.000000,0.714286,-0.989259,-2.063061,...,0.241745,0.055830,-0.341265,0.496525,0.066970,-0.371546,0.474984,-0.257203,-0.731698,0.211711
66%,37119.000000,-24.000000,-14.000000,-3.000000,70.000000,13.000000,-31.000000,4.166667,-0.228442,-1.342826,...,0.620089,0.364892,-0.036693,0.823879,0.337154,-0.069670,0.754070,-0.016043,-0.474995,0.492962
max,55133.000000,141.000000,25.000000,62.000000,658.000000,38.000000,8.000000,1594.142857,8.472449,4.212593,...,11.113996,3.129203,2.598948,8.735262,2.851894,2.403462,15.844432,3.286648,2.725306,10.287810


In [11]:
N_DAYS = 7
d2_threshold = 6.0
data_x = []
data_y = []
x_cols = []
#x_cols = ["mobility_transit_stations","mobility_retail_and_recreation","mobility_grocery_and_pharmacy","mobility_parks","mobility_residential","mobility_workplaces"] # Mobility data
for i in range(11): # CAMS data
    x_cols.extend(["x{:n}_mean".format(i)]) #"x{:n}_mean".format(i),"x{:n}_amin".format(i),

x_labels = []

for group,group_df in df.groupby(by=["shapeID"]):
    dates = group_df["date"]
    x = group_df[x_cols].values
    y = group_df["smoothed_d2"].values
    for i in range(len(x)-7):
        if (dates.iloc[i+6] - dates.iloc[i]).days == N_DAYS - 1:
            x_labels.append((str(group),dates.iloc[i],dates.iloc[i+6]))
            data_x.append(x[i:i+7].flatten())
            y_period = y[i:i+7]
            data_y.append([np.sum(y_period < -d2_threshold),np.sum(np.abs(y_period) <= d2_threshold),np.sum(y_period > d2_threshold)])
        else:
            i += 6

data_x = np.array(data_x)
data_y = np.array(data_y) # All equal to 7
#print(len(data_x),len(data_y),np.sum(np.sum(data_y,axis=1) != 7))
x_labels = np.array(x_labels)

data_y = data_y.astype(float) / 7.0


class_references = [
    -np.log(np.array([ # Decelerating
        [0.98,0.01,0.01],
        [0.7,0.2,0.1]
    ])),
    -np.log(np.array([ # Stable
        [0.33,0.34,0.33],
        [0.495,0.01,0.495],
        [0.1,0.8,0.1],
        [0.01,0.98,0.01]
    ])),
    -np.log(np.array([ # Accelerating
        [0.01,0.01,0.98],
        [0.1,0.2,0.7]
    ]))
]

# Minimize cross-entropy of class reference distributions 
data_y_mapped = np.array(list(map(lambda x : np.argmin([np.min(np.sum(x*class_references[i],axis=1)) for i in range(len(class_references))]),data_y)))
#print(data_y[data_y_mapped == 0][:10])
#print(data_y[data_y_mapped == 1][:10])
#print(data_y[data_y[:,1] < 0.15][:10])
#print(data_y_mapped[data_y[:,1] < 0.15][:10])
#print(data_y[data_y_mapped == 2][:10])
print(len(data_y_mapped[data_y_mapped == 0])/len(data_y_mapped),len(data_y_mapped[data_y_mapped == 1])/len(data_y_mapped),len(data_y_mapped[data_y_mapped == 2])/len(data_y_mapped))

dat = np.arange(len(data_y_mapped))
decel = dat[data_y_mapped == 0]
accel = dat[data_y_mapped == 2]
np.random.shuffle(accel)
accel = accel[:len(decel)]
stabl = dat[data_y_mapped == 1]
np.random.shuffle(stabl)
stabl = stabl[:len(decel)]

dat = np.append(decel,np.append(stabl,accel))
np.random.shuffle(dat)

data_x = data_x[dat]
data_y_mapped = data_y_mapped[dat]
x_labels = x_labels[dat]

print(len(data_y_mapped[data_y_mapped == 0])/len(data_y_mapped),len(data_y_mapped[data_y_mapped == 1])/len(data_y_mapped),len(data_y_mapped[data_y_mapped == 2])/len(data_y_mapped))
print(len(data_y_mapped))
print(len(x_labels))

0.08274026858097655 0.7531716071539081 0.1640881242651154
0.3333333333333333 0.3333333333333333 0.3333333333333333
8022
8022


In [56]:
if np.sum(data_x != data_x) > 0: # Count NaNs
    raise Exception("This data contains NaNs.")

```
ids = df["shapeID"].drop_duplicates().values

N_DAYS = 7
times = df["time"].drop_duplicates().values
times_grouped = [times[n:n+N_DAYS] for n in range(0, len(times), N_DAYS)]
times_grouped.pop() # not len 14

cols = df.drop(['time', 'shapeID', 'smoothed_d2'], axis=1).columns
grouped_cols = []
for i in range(0, N_DAYS):
    for label in cols:
        grouped_cols.append('Day {}: {}'.format(i, label))

X_grouped = pd.DataFrame(columns=grouped_cols)
Y_grouped = pd.DataFrame(columns=['y{}'.format(i) for i in range(0,N_DAYS)])

errors = []
bad_format_error = 0
for id in ids:
    for dates in times_grouped:

        id_query = df.shapeID == id
        date_query = df.time.isin(dates)
        query = df[id_query & date_query].drop(['time', 'shapeID'], axis=1)
        
        x_vals = query.drop('smoothed_d2', axis=1).values.flatten()
        y_vals = query.smoothed_d2.values.flatten()

        if(len(x_vals)) != len(grouped_cols):
            bad_format_error+=1
            errors.append(query)
        else:
            df_x_temp = pd.DataFrame(x_vals.reshape(1,-1), columns=grouped_cols)
            df_y_temp = pd.DataFrame(y_vals.reshape(1,-1), columns=['y{}'.format(i) for i in range(0,N_DAYS)])
            X_grouped = X_grouped.append(df_x_temp, ignore_index=True)
            Y_grouped = Y_grouped.append(df_y_temp, ignore_index=True)
print("SUCCESS with {}/{} FORMAT_ERRORS".format(bad_format_error, len(ids) * len(times_grouped)))
X_grouped
```

```
N_PCs = 11
avg_cols = ["x{}_mean".format(i) for i in range(0,N_PCs)]
min_cols = ["x{}_amin".format(i) for i in range(0,N_PCs)]
max_cols = ["x{}_amax".format(i) for i in range(0,N_PCs)]
generic_cols = ['time', 'shapeID', 'mobility_transit_stations', 'mobility_retail_and_recreation',
       'mobility_grocery_and_pharmacy', 'mobility_parks',
       'mobility_residential', 'mobility_workplaces']

def get_df_from_cols(dataframe, cols):
    X_new = pd.DataFrame()
    for column in dataframe.columns:
        for label in cols:
            if label in column:
                X_new[column] = dataframe[column]
    return X_new

X_no_cams = get_df_from_cols(X_grouped, generic_cols)
X_min = get_df_from_cols(X_grouped, min_cols+generic_cols)
X_max = get_df_from_cols(X_grouped, max_cols+generic_cols)
X_avg = get_df_from_cols(X_grouped, avg_cols+generic_cols)
```

In [41]:
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.ensemble import AdaBoostRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

X_train = X_no_cams.to_numpy()
y_train = Y_grouped.mean().to_numpy()

num_folds = 10
seed = 7
scoring = 'neg_mean_squared_error'

models = []
models.append(('GB', GradientBoostingRegressor()))
models.append(('AB', AdaBoostRegressor()))
models.append(('DT', DecisionTreeRegressor()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('LR', LinearRegression()))

results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s %f %f " % (name, cv_results.mean(), cv_results.std())
    print(msg)

DT -2937.549588 1352.933767 
KNN -1483.987277 1304.749986 
LR -1290.123501 1250.325647 


Results: no difference

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data_x,data_y_mapped,test_size=0.2)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

#TRESHOLD = 10

#X_train = X_min.to_numpy()
#y_train = (Y_grouped > TRESHOLD).sum(axis=1)

num_folds = 10
seed = 7
scoring = 'accuracy'

models = []
models.append(('GB', GradientBoostingClassifier()))
models.append(('AB', AdaBoostClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('KNN5', KNeighborsClassifier()))
#models.append(('KNN6', KNeighborsClassifier(n_neighbors=6)))
#models.append(('KNN7', KNeighborsClassifier(n_neighbors=7)))
#models.append(('KNN8', KNeighborsClassifier(n_neighbors=8)))
#models.append(('KNN9', KNeighborsClassifier(n_neighbors=9)))
#models.append(('KNN10', KNeighborsClassifier(n_neighbors=10)))
models.append(('LR', LogisticRegression()))

results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s %f %f " % (name, cv_results.mean(), cv_results.std())
    print(msg)

print(results)

GB 0.530381 0.012139 
AB 0.488989 0.014139 
DT 0.472614 0.015494 
KNN5 0.560126 0.013314 
LR 0.489331 0.012373 
[array([0.54262673, 0.54493088, 0.50576037, 0.53056517, 0.53402537,
       0.5455594 , 0.52018454, 0.53517878, 0.52018454, 0.52479815]), array([0.50576037, 0.48502304, 0.47235023, 0.46366782, 0.49134948,
       0.50865052, 0.47981546, 0.50634371, 0.48673587, 0.49019608]), array([0.47235023, 0.50115207, 0.46428571, 0.45098039, 0.45790081,
       0.49480969, 0.46597463, 0.48442907, 0.46020761, 0.47404844]), array([0.55760369, 0.57488479, 0.57834101, 0.55594002, 0.55363322,
       0.57093426, 0.544406  , 0.57670127, 0.54094579, 0.54786621]), array([0.50806452, 0.50345622, 0.48732719, 0.49250288, 0.50288351,
       0.48788927, 0.48558247, 0.47174164, 0.46828143, 0.48558247])]


Mobility :
- GB 0.521731 0.020419 
- AB 0.464854 0.021199 
- DT 0.445842 0.014300 
- KNN 0.477953 0.011020 
- LR 0.433846 0.017263


Mobility + CAMS (mean) : 
- GB 0.585315 0.021577 
- AB 0.523920 0.016988 
- DT 0.503976 0.022049 
- KNN 0.482482 0.025346 
- LR 0.524232 0.022335


CAMS only:
- GB 0.570983 0.017125 
- AB 0.498513 0.016178 
- DT 0.491812 0.013192 
- KNN 0.592799 0.012372 
- LR 0.500387 0.011376


CAMS only (first 6 PCs):
- GB 0.553225 0.022999 
- AB 0.489639 0.019175 
- DT 0.479038 0.014877 
- KNN 0.575815 0.013192 
- LR 0.503043 0.011353


CAMS only (max only):
- GB 0.572539 0.016715 
- AB 0.505224 0.015899 
- DT 0.490577 0.028445 
- KNN 0.591241 0.013768 
- LR 0.503973 0.018374


CAMS only (min only):
- GB 0.558206 0.012808 
- AB 0.499603 0.013846 
- DT 0.492600 0.011840 
- KNN 0.588752 0.019825 
- LR 0.505990 0.017910


CAMS only (mean only):
- GB 0.559451 0.018181 
- AB 0.507402 0.013728 
- DT 0.495705 0.027473 
- KNN 0.597318 0.014909 
- LR 0.512234 0.018876


CAMS only (mean only):

k run 1 :
- KNN5 0.610874 0.019984 
- KNN10 0.607912 0.009555 
- KNN15 0.606355 0.011807 
- KNN20 0.605424 0.012652 
- KNN30 0.604021 0.012731

k run 2 :
- KNN5 0.596858 0.020018 
- KNN6 0.596081 0.015506 
- KNN7 0.601533 0.017145 
- KNN8 0.599196 0.013899 
- KNN9 0.597012 0.009711 
- KNN10 0.595767 0.012003

MEAN OF WEEK W HIGHER 10 CLASSIFICATION

- GB 0.927965 0.019642 
- AB 0.925721 0.020152 
- DT 0.875234 0.026127 
- KNN 0.924821 0.022831 
- LR 0.927741 0.019161 

(w cams)
- GB 0.927293 0.020238 
- AB 0.923704 0.018885 
- DT 0.875671 0.011897 
- KNN 0.923250 0.023082 
- LR 0.922804 0.018741 



N DAYS HIGHER 10 CLASSIFICATION

- GB 0.450636 0.063706 
- AB 0.430657 0.074770 
- DT 0.319354 0.038397 
- KNN 0.410910 0.041697 
- LR 0.460284 0.067121 

(w cams)
- GB 0.455341 0.057830 
- AB 0.442550 0.058166 
- DT 0.346043 0.033837 
- KNN 0.412478 0.039655 
- LR 0.447716 0.061701 

In [12]:
#print(len(data_y_mapped))
# Regional splitting
regions = {}
west_mask = ["6","32","41","53"]
east_mask = ["9","10","11","23","24","25","34","36","37","42","44","50","51"]
south_mask = ["1","5","8","22","35","48"]
east_nomask = ["12","13","45"]
middle_mask = ["17","18","21","26","27","39","54","55"]
middle_nomask = ["16","19","28","29","31","38","40","46","47","56"]

splits = []
west_mask_split = []
east_mask_split = []
south_mask_split = []
east_nomask_split = []
middle_mask_split = []
middle_nomask_split = []

for i in range(len(x_labels)):
    state_fips = x_labels[i][0][:-3]
    if state_fips in west_mask:
        west_mask_split.append(i)
    elif state_fips in east_mask:
        east_mask_split.append(i)
    elif state_fips in south_mask:
        south_mask_split.append(i)
    elif state_fips in east_nomask:
        east_nomask_split.append(i)
    elif state_fips in middle_mask:
        middle_mask_split.append(i)
    elif state_fips in middle_nomask:
        middle_nomask_split.append(i)

west_mask_split = np.array(west_mask_split)
east_mask_split = np.array(east_mask_split)
south_mask_split = np.array(south_mask_split)
east_nomask_split = np.array(east_nomask_split)
middle_mask_split = np.array(middle_mask_split)
middle_nomask_split = np.array(middle_nomask_split)

def display_split_proportions(mask):
    tmp = data_y_mapped[mask]
    print(np.average(tmp == 0),np.average(tmp == 1),np.average(tmp == 2),len(tmp))

display_split_proportions(west_mask_split)
display_split_proportions(east_mask_split)
display_split_proportions(south_mask_split)
display_split_proportions(east_nomask_split)
display_split_proportions(middle_mask_split)
display_split_proportions(middle_nomask_split)

0.1702127659574468 0.475177304964539 0.3546099290780142 141
0.34138785625774476 0.35811648079306074 0.30049566294919455 3228
0.3674418604651163 0.2651162790697674 0.3674418604651163 860
0.4333868378812199 0.3073836276083467 0.2592295345104334 1246
0.25 0.3398809523809524 0.4101190476190476 1680
0.35240963855421686 0.2680722891566265 0.3795180722891566 664


In [23]:
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt

def plot_confusion_matrix(classes,y_pred,y,savepath=None):
    if len(y) != len(y_pred):
        raise Exception("Truth and prediction do not have equal length.")
    class_index = {}
    for i in range(len(classes)):
        class_index[classes[i]] = i
    
    mat = np.zeros((len(classes),len(classes)))
    for i in range(len(y)):
        mat[class_index[y[i]]][class_index[y_pred[i]]] += 1
    
    for i in range(len(mat)):
        mat[i] /= np.sum(mat[i])

    fig, ax = plt.subplots()
    im = ax.imshow(mat,cmap="rainbow",vmin=0.0,vmax=1.0)

    ax.set_xticks(range(len(classes)))
    ax.set_yticks(range(len(classes)))
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)

    # Rotate the tick labels and set their alignment.
    #plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
    #        rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    #for i in y:
    #    for j in x:
    #        text = ax.text(j, i, "{:.4f}".format(mat[i,j]),
    #                    ha="center", va="center", color="black",fontsize=8)

    #fig.colorbar(im)

    ax.set_xlabel("Predicted class")
    ax.set_ylabel("True class")

    fig.colorbar(im)

    fig.tight_layout()
    if savepath is None:
        plt.show()
    else:
        plt.savefig(savepath)
    plt.close()

classifier1 = KNeighborsClassifier()
classifier2 = GradientBoostingClassifier()

pos = set(range(len(data_y_mapped)))

totLen = len(east_mask_split) + len(west_mask_split) + len(south_mask_split) + len(east_nomask_split) + len(middle_mask_split) + len(middle_nomask_split)

weights = []
accuracy = []
accuracy_r = []

def classify_and_display_results(x_train,y_train,x_test,y_test,classifier,outName):
    classifier.fit(x_train,y_train)
    y_pred = classifier.predict(x_test)

    classifier.fit(x_test,y_test)
    y_pred_r = classifier.predict(x_train)
    a = np.average(y_test == y_pred)
    a_r = np.average(y_train == y_pred_r)
    accuracy.append(a)
    accuracy_r.append(a_r)
    weights.append(len(x_train)/totLen)
    print(outName,a,a_r)
    plot_confusion_matrix([0,1,2],y_pred,y_test,savepath=outName + ".png")
    plot_confusion_matrix([0,1,2],y_pred_r,y_train,savepath=outName + "_r.png")


totLen = len(east_mask_split) + len(west_mask_split) + len(south_mask_split) + len(east_nomask_split) + len(middle_mask_split) + len(middle_nomask_split)

for name,split in (("west_mask",west_mask_split),("east_mask",east_mask_split),("south_mask",south_mask_split),("east_no_mask",east_nomask_split),("middle_mask",middle_mask_split),("middle_no_mask",middle_nomask_split)):
    pos_train = split
    pos_test = np.array(list(pos.difference(set(split))))

    #print(pos_train[:10])
    #print(pos_test[:10])

    x_train = data_x[pos_train]
    y_train = data_y_mapped[pos_train]
    x_test = data_x[pos_test]
    y_test = data_y_mapped[pos_test]

    classify_and_display_results(x_train,y_train,x_test,y_test,classifier1,name + "_knn")
    classify_and_display_results(x_train,y_train,x_test,y_test,classifier2,name + "_gbc")

mean_knn = np.average(accuracy[::2],weights=weights[::2])
mean_knn_r = np.average(accuracy_r[::2],weights=weights[::2])
mean_gbc = np.average(accuracy[1::2],weights=weights[1::2])
mean_gbc_r = np.average(accuracy_r[1::2],weights=weights[1::2])

print(mean_knn,np.sqrt(np.average((np.array(accuracy[::2]) - mean_knn)**2,weights=weights[::2])))
print(mean_knn_r,np.sqrt(np.average((np.array(accuracy_r[::2]) - mean_knn_r)**2,weights=weights[::2])))
print(mean_gbc,np.sqrt(np.average((np.array(accuracy[1::2]) - mean_gbc)**2,weights=weights[1::2])))
print(mean_gbc_r,np.sqrt(np.average((np.array(accuracy_r[1::2]) - mean_gbc_r)**2,weights=weights[1::2])))

west_mask_knn 0.43547773125237915 0.3900709219858156
west_mask_gbc 0.37228778073848495 0.524822695035461
east_mask_knn 0.4213600333750521 0.5086741016109045
east_mask_gbc 0.4025865665415102 0.49876084262701365
south_mask_knn 0.40589220888020106 0.3872093023255814
south_mask_gbc 0.3821558223959788 0.3883720930232558
east_no_mask_knn 0.34371310507674147 0.37640449438202245
east_no_mask_gbc 0.3572904368358914 0.3996789727126806
middle_mask_knn 0.4495427309996846 0.5238095238095238
middle_mask_gbc 0.4421318196152633 0.5172619047619048
middle_no_mask_knn 0.41709703723838 0.4292168674698795
middle_no_mask_gbc 0.4149225332970916 0.37349397590361444
0.4132332263984654 0.033118352525934334
0.46860212303363596 0.06036529300484583
0.4021191871724127 0.027168577630856155
0.46463742166517463 0.056201990661977015


Predictions using mobility only
- west_mask_knn 0.3860695255011418 0.24285714285714285
- west_mask_gbc 0.359172798782035 0.32142857142857145
- east_mask_knn 0.3784679089026915 0.4467418546365915
- east_mask_gbc 0.4062111801242236 0.47775689223057644
- south_mask_knn 0.38040587823652905 0.3751425313568985
- south_mask_gbc 0.3885234429671099 0.40022805017103763
- east_no_mask_knn 0.37196206283343214 0.4050235478806907
- east_no_mask_gbc 0.40411973918197985 0.34301412872841447
- middle_mask_knn 0.38682084587359417 0.3856056173200702
- middle_mask_gbc 0.4268968794550927 0.5002925687536571
- middle_no_mask_knn 0.3321564367191744 0.3860182370820669
- middle_no_mask_gbc 0.3661053775122216 0.36322188449848025

Predictions using mobility + CAMS (mean only):
- west_mask_knn 0.4028941355674029 0.3125
- west_mask_gbc 0.39756283320639757 0.4513888888888889
- east_mask_knn 0.3752069536423841 0.43855799373040755
- east_mask_gbc 0.4236341059602649 0.5360501567398119
- south_mask_knn 0.3610411418975651 0.3641552511415525
- south_mask_gbc 0.382031905961377 0.3550228310502283
- east_no_mask_knn 0.38164538164538164 0.3781055900621118
- east_no_mask_gbc 0.41847341847341846 0.41770186335403725
- middle_mask_knn 0.3962501969434378 0.39761194029850744
- middle_mask_gbc 0.4810146525917756 0.5438805970149254
- middle_no_mask_knn 0.3484992530218661 0.3899848254931715
- middle_no_mask_gbc 0.4101589026212142 0.40212443095599393

Predictions using CAMS mean :
- west_mask_knn 0.41686746987951806 0.46715328467153283
- west_mask_gbc 0.4040583386176284 0.44525547445255476
- east_mask_knn 0.4161825726141079 0.5237351655215491
- east_mask_gbc 0.4099585062240664 0.5031230480949407
- south_mask_knn 0.4073504751257686 0.3810623556581986
- south_mask_gbc 0.38038010061486865 0.34295612009237875
- east_no_mask_knn 0.38438749814842244 0.3878835562549174
- east_no_mask_gbc 0.3901644200859132 0.3870967741935484
- middle_mask_knn 0.4542020774315392 0.526378896882494
- middle_mask_gbc 0.4446018256216557 0.5449640287769785
- middle_no_mask_knn 0.41800380745172694 0.4565868263473054
- middle_no_mask_gbc 0.41011694316018493 0.41916167664670656

Predictions using CAMS max :
- west mask 0.42224759005580925 0.39855072463768115
- east mask 0.406575114440283 0.47947761194029853
- south mask 0.385 0.40389294403892945
- east no mask 0.37166567871962064 0.39089481946624804
- middle mask 0.4332118523213437 0.5160724722384571
- middle no mask 0.3960059774487162 0.43872919818456885

Predictions using CAMS min :
- west mask 0.402000506457331 0.4032258064516129
- east mask 0.40731504571903576 0.4853582554517134
- south mask 0.3862396867570969 0.39380022962112515
- east no mask 0.35011848341232227 0.3409448818897638
- middle mask 0.43080321918889064 0.5145400593471811
- middle no mask 0.42363190852164445 0.4378698224852071

Predictions using CAMS max and min :
- west mask 0.43273372181403597 0.4296875
- east mask 0.4157372986369269 0.49937106918238994
- south mask 0.38004484304932734 0.40970654627539504
- east no mask 0.3429377691964949 0.37548487199379366
- middle mask 0.45731031205449074 0.521357519016969
- middle no mask 0.4195690472963816 0.4510108864696734

In [19]:
N_DAYS = 7
d2_threshold = 6.0
data_x = []
data_y = []
x_cols = []
#x_cols = ["mobility_transit_stations","mobility_retail_and_recreation","mobility_grocery_and_pharmacy","mobility_parks","mobility_residential","mobility_workplaces"] # Mobility data
for i in range(11): # CAMS data
    x_cols.extend(["x{:n}_mean".format(i)]) #"x{:n}_mean".format(i),"x{:n}_amin".format(i),

x_labels = []

for group,group_df in df.groupby(by=["shapeID"]):
    dates = group_df["date"]
    x = group_df[x_cols].values
    y = group_df["smoothed_d2"].values
    for i in range(len(x)-7):
        if (dates.iloc[i+6] - dates.iloc[i]).days == N_DAYS - 1:
            x_labels.append((group,dates.iloc[i],dates.iloc[i+6]))
            data_x.append(x[i:i+7].flatten())
            y_period = y[i:i+7]
            data_y.append([np.sum(y_period < -d2_threshold),np.sum(np.abs(y_period) <= d2_threshold),np.sum(y_period > d2_threshold)])
        else:
            i += 6

data_x = np.array(data_x)
data_y = np.array(data_y) # All equal to 7
#print(len(data_x),len(data_y),np.sum(np.sum(data_y,axis=1) != 7))

data_y = data_y.astype(float) / 7.0


class_references = [
    -np.log(np.array([ # Decelerating
        [0.98,0.01,0.01],
        [0.7,0.2,0.1]
    ])),
    -np.log(np.array([ # Stable
        [0.33,0.34,0.33],
        [0.495,0.01,0.495],
        [0.1,0.8,0.1],
        [0.01,0.98,0.01]
    ])),
    -np.log(np.array([ # Accelerating
        [0.01,0.01,0.98],
        [0.1,0.2,0.7]
    ]))
]

# Minimize cross-entropy of class reference distributions 
data_y_mapped = np.array(list(map(lambda x : np.argmin([np.min(np.sum(x*class_references[i],axis=1)) for i in range(len(class_references))]),data_y)))
#print(data_y[data_y_mapped == 0][:10])
#print(data_y[data_y_mapped == 1][:10])
#print(data_y[data_y[:,1] < 0.15][:10])
#print(data_y_mapped[data_y[:,1] < 0.15][:10])
#print(data_y[data_y_mapped == 2][:10])
print(len(data_y_mapped[data_y_mapped == 0])/len(data_y_mapped),len(data_y_mapped[data_y_mapped == 1])/len(data_y_mapped),len(data_y_mapped[data_y_mapped == 2])/len(data_y_mapped))

dat = np.arange(len(data_y_mapped))
decel = dat[data_y_mapped == 0]
accel = dat[data_y_mapped == 2]
np.random.shuffle(accel)
accel = accel[:len(decel)]
stabl = dat[data_y_mapped == 1]
np.random.shuffle(stabl)
stabl = stabl[:len(decel)]

dat = np.append(decel,np.append(stabl,accel))
np.random.shuffle(dat)

data_x_train = data_x[dat]
data_y_mapped_train = data_y_mapped[dat]

0.08274026858097655 0.7531716071539081 0.1640881242651154


In [20]:
classifier = KNeighborsClassifier(n_neighbors=7)
classifier.fit(data_x_train,data_y_mapped_train)

y_pred = classifier.predict(data_x)
print(np.sum(y_pred == data_y_mapped)/len(data_y))

0.0


In [37]:
f = open("prediction_results.csv","w")

for i in range(len(x_labels)):
    f.write(str(x_labels[i][0]) + ","
            + str(x_labels[i][1].year) + "-" + "{:02n}".format(x_labels[i][1].month) + "-" + "{:02n}".format(x_labels[i][1].day) + ","
            + str(x_labels[i][2].year) + "-" + "{:02n}".format(x_labels[i][2].month) + "-" + "{:02n}".format(x_labels[i][2].day) + ","
            + str(y_pred[i]) + "," + str(data_y_mapped[i]) + "\n")

f.close()

In [32]:
x_labels[i][1].year

2020